In [1]:
base_kufi_path = "warsh_ibrahim_aldosary_wav_files"
base_madani_path = "warsh_ibrahim_aldosary_wav_files_madani_hisb_60_and_Al_Fatihah"

In [2]:
import wave
import os

In [3]:
os.makedirs(base_madani_path, exist_ok = True)

In [4]:
def combine_wav(files, output_file):
    with wave.open(files[0], 'rb') as first_file:
        params = first_file.getparams()
        with wave.open(output_file, 'wb') as output:
            output.setparams(params)
            for file in files:
                with wave.open(file, 'rb') as input_file:
                    output.writeframes(input_file.readframes(input_file.getnframes()))

In [5]:
def split_wav(input_file, output_dir, new_ayat, split_points):
    with wave.open(input_file, 'rb') as input_wav:
        params = input_wav.getparams()
        frame_rate = input_wav.getframerate()
        total_frames = input_wav.getnframes()
        split_frames = [int(point * frame_rate) for point in split_points]
        split_frames.append(total_frames)
        start_frame = 0
        for i, split_frame in enumerate(split_frames):
            frames_to_read = split_frame - start_frame
            frames = input_wav.readframes(frames_to_read)
            output_file = os.path.join(output_dir, f'{new_ayat[i]}.wav')
            with wave.open(output_file, 'wb') as output_wav:
                output_wav.setparams(params)
                output_wav.writeframes(frames)
            start_frame = split_frame

In [6]:
ayat_to_combine = [ 
# ["sura", ["ayat to combine"], "new aya"],
    # Al-Fajr
    ["089", ["029", "030"], "032"],
    # Al-Qari'ah
    ["101", ["001", "002"], "001"],
    # Al-Asr
    ["103", ["001", "002"], "001"],
    # Al-Ma'un
    ["107", ["006", "007"], "006"]
]

In [7]:
for surah, ayat, new_ayah in ayat_to_combine:
        surah_directory = os.path.join(base_kufi_path, surah)
        output_file = f"{base_madani_path}/{surah}/{new_ayah}.wav"
        wav_files = [os.path.join(surah_directory, f"{ayat_number}.wav") for ayat_number in ayat]
        surah_new_path = os.path.join(base_madani_path, surah) 
        os.makedirs(surah_new_path, exist_ok = True)
        combine_wav(wav_files, output_file)

In [8]:
ayat_to_split = [ 
# ["sura", "aya", ["new ayat"], [split at seconds]],
    # Al-Fatiha
    ["001", "007", ["006", "007"], [5]],
    # Al-Fajr
    ["089", "015", ["015", "016"], [10]],
    ["089", "016", ["017", "018"], [9]],
    ["089", "023", ["025", "026"], [8]],
    # Al-Alaq
    ["096", "015", ["015", "016"], [4]],
    # Az-Zalzalah
    ["099", "006", ["006", "007"], [6]],
    # Al-Asr
    ["103", "003", ["002", "003"], [10]],
    # Quraysh
    ["106", "004", ["004", "005"], [9]]
]

In [9]:
for surah, ayah, new_ayat, split_points in ayat_to_split:
        surah_dir = os.path.join(base_kufi_path, surah)
        input_file = os.path.join(surah_dir, f'{ayah}.wav')
        surah_new_path = os.path.join(base_madani_path, surah) 
        os.makedirs(surah_new_path, exist_ok = True)
        split_wav(input_file, surah_new_path, new_ayat, split_points)

In [10]:
ayat_to_skip = {
    # surah : [ayat to skip]
    "001": ["007"],
    "089": ["015", "016", "023", "029", "030"],
    "096": ["015"],
    "099": ["006"],
    "101": ["001", "002"],
    "103": ["001", "002", "003"],
    "106": ["004"],
    "107": ["006", "007"]
}

In [11]:
import shutil

In [12]:
new_ayat = {
    "001": ["006", "007"],
    "089": ["015", "016", "017", "018", "025", "026", "032"],
    "096": ["015", "016"],
    "099": ["006", "007"],
    "101": ["001"],
    "103": ["001", "002", "003"],
    "106": ["004", "005"],
    "107": ["006"]
}

In [13]:
def convert_to_three_digit_string(number):
    return str(number).zfill(3)

In [14]:
def copy_and_rename_files(kufi_source_dir, destination_dir):
    i = 1
    for filename in os.listdir(kufi_source_dir):
        _, surah = kufi_source_dir.split("/")
        aya = filename.split(".")[0]
        if surah == "001" and aya == "001":
            continue
        if aya not in ayat_to_skip.get(surah, []):
            new_aya_name = convert_to_three_digit_string(i)
            while(new_aya_name in new_ayat.get(surah,[])):
                i+= 1
                new_aya_name = convert_to_three_digit_string(i)
            new_filename = f"{new_aya_name}.wav" 
            shutil.copy(os.path.join(kufi_source_dir, filename), os.path.join(destination_dir, new_filename))
            i+= 1

In [15]:
for i in range(1, 2):
    dirname = str(i).zfill(3)
    os.makedirs(base_madani_path, exist_ok = True)
    destination_dir = os.path.join(base_madani_path, dirname) 
    os.makedirs(destination_dir, exist_ok = True)
    copy_and_rename_files(f'{base_kufi_path}/{dirname}', destination_dir)

In [16]:
for i in range(87, 115):
    dirname = str(i).zfill(3)
    os.makedirs(base_madani_path, exist_ok = True)
    destination_dir = os.path.join(base_madani_path, dirname) 
    os.makedirs(destination_dir, exist_ok = True)
    copy_and_rename_files(f'{base_kufi_path}/{dirname}', destination_dir)

In [17]:
def count_files_in_directory_w(directory):
    files = os.listdir(directory)
    return len(files)

In [18]:
total_ayat = 0 
for dirname in os.listdir(base_madani_path):
    total_ayat += count_files_in_directory_w(f'{base_madani_path}/{dirname}')
total_ayat

298